# Predire il sentimento dalle review dei prodotti 



In [1]:
from __future__ import division
import graphlab
import math
import string

# Data preperation



In [21]:
# Scommentare la riga sotto per caricare il dataset
#
#products = graphlab.SFrame.read_csv('data/Reviews.csv')

Now, let us see a preview of what the dataset looks like.

In [3]:
products

Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5
2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1
3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""NataliaCorres"" ...",1,1,4
4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2
5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M.Wassir"" ...",0,0,5
6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4
7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5
8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5
9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5
10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5


## Build the word count vector for each review

Vediamo uno specifico esempio di prodotto

In [4]:
products[12]

{'HelpfulnessDenominator': 1,
 'HelpfulnessNumerator': 1,
 'Id': 13,
 'ProductId': 'B0009XLVG0',
 'ProfileName': 'LT',
 'Score': 1,
 'Summary': 'My Cats Are Not Fans of the New Food',
 'Text': "My cats have been happily eating Felidae Platinum for more than two years. I just got a new bag and the shape of the food is different. They tried the new food when I first put it in their bowls and now the bowls sit full and the kitties will not touch the food. I've noticed similar reviews related to formula changes in the past. Unfortunately, I now need to find a new food that my cats will eat.",
 'Time': 1339545600,
 'UserId': 'A327PCT23YH90'}

Ora faremo due semplici operazioni:
* Rimozione della punteggiatura [Python's built-in](https://docs.python.org/2/library/string.html)
* Word Count.


In [6]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_puctuation = products['Text'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_puctuation)
products['word_count'][12]

{'a': 2,
 'and': 3,
 'bag': 1,
 'been': 1,
 'bowls': 2,
 'cats': 2,
 'changes': 1,
 'different': 1,
 'eat': 1,
 'eating': 1,
 'felidae': 1,
 'find': 1,
 'first': 1,
 'food': 4,
 'for': 1,
 'formula': 1,
 'full': 1,
 'got': 1,
 'happily': 1,
 'have': 1,
 'i': 3,
 'in': 2,
 'is': 1,
 'it': 1,
 'ive': 1,
 'just': 1,
 'kitties': 1,
 'more': 1,
 'my': 2,
 'need': 1,
 'new': 3,
 'not': 1,
 'noticed': 1,
 'now': 2,
 'of': 1,
 'past': 1,
 'platinum': 1,
 'put': 1,
 'related': 1,
 'reviews': 1,
 'shape': 1,
 'similar': 1,
 'sit': 1,
 'than': 1,
 'that': 1,
 'the': 7,
 'their': 1,
 'they': 1,
 'to': 2,
 'touch': 1,
 'tried': 1,
 'two': 1,
 'unfortunately': 1,
 'when': 1,
 'will': 2,
 'years': 1}

In [14]:
graphlab.text_analytics.stopwords()

In [7]:
review_without_stopwords = products['word_count'].dict_trim_by_keys(graphlab.text_analytics.stopwords(), True)
products['filter_stopwords'] = review_without_stopwords


Vediamo un esempio dopo queste due trasformazioni. Ogni entry della colonna **filter_stopwords** è un dizionario dove la chiave è la parola e il valore è il conteggio del numero di volte che la parola occorre. 

In [8]:
products[12]['filter_stopwords']

{'bag': 1,
 'bowls': 2,
 'cats': 2,
 'eat': 1,
 'eating': 1,
 'felidae': 1,
 'find': 1,
 'food': 4,
 'formula': 1,
 'full': 1,
 'happily': 1,
 'ive': 1,
 'kitties': 1,
 'noticed': 1,
 'past': 1,
 'platinum': 1,
 'put': 1,
 'related': 1,
 'reviews': 1,
 'shape': 1,
 'similar': 1,
 'sit': 1,
 'touch': 1,
 'years': 1}

## Estrazione dei sentimenti

Ignoreremo tutte le reviews che hanno * rating =3 *, perchè tendono ad avere sentimento neutrale.  

In [9]:
products = products[products['Score'] != 3]
len(products)

525812

* Assegniamo un sentimento positivo a tutte le reviews che hanno uno score >=4 e un sentimento negativo a tutte quelle che hanno un voto pari a 2 o più basso.
* Utilizziamo +1 per sentimento positivo e -1 per quello negativo, come etichette delle classi.

In [10]:
products['sentiment'] = products['Score'].apply(lambda rating : +1 if rating > 3 else -1)

Ora la colonna contiene una colonna extra chiamata  **sentimento** che è o positiva (+1) o negativa(-1).

## Train e test set

Facciamo una divisione train/test dove l'80% dei dati è all'interno del training set e il 20% dei dati è nel test set. Utilizziamo seed=0, in modo da ottenere sempre lo stesso risultato.

In [11]:
train_data, test_data = products.random_split(.8, seed=0)
print len(train_data)
print len(test_data)

420615
105197


# Addestra un sentiment classifier con la logistic  regression.  

Utilizziamo la logistic regression per creare un sentiment classifier sui dati di training.  In questo modello utilizzeremo la colonna stopwords come caratteristica e la colonna sentiment come target.   

**NB:** Può metterci qualche minuto. 

In [15]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['filter_stopwords'],
                                                      validation_set=None)

<pre>Logistic regression:</pre>

<pre>--------------------------------------------------------</pre>

<pre>Number of examples          : 420615</pre>

<pre>Number of classes           : 2</pre>

<pre>Number of feature columns   : 1</pre>

...

<pre>| 9         | 16       | 1.000000  | 15.197262    | 0.970084          |</pre>

<pre>| 10        | 17       | 1.000000  | 16.314656    | 0.972374          |</pre>

<pre>+-----------+----------+-----------+--------------+-------------------+</pre>

<pre>TERMINATED: Iteration limit reached.</pre>

<pre>This model may not be optimal. To improve it, consider increasing `max_iterations`.</pre>

Una volta addestrato il modello è possibile vedere i coefficienti.

In [16]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

In [17]:
weights

name,index,class,value,stderr
(intercept),None,1,0.904186705898,None
filter_stopwords,appreciates,1,0.660950962771,None
filter_stopwords,labrador,1,0.077241980261,None
filter_stopwords,smells,1,-0.14982514128,None
filter_stopwords,meat,1,0.0165779738554,None
filter_stopwords,processed,1,-0.146814127141,None
filter_stopwords,stew,1,0.378883626945,None
filter_stopwords,bought,1,-0.355416224029,None
filter_stopwords,product,1,-0.0521112837281,None
filter_stopwords,finicky,1,0.598182320732,None


Ci sono un totale di `213505` coefficienti nel modello. Ricorda dalla lezione precedente : i pesi positivi  $w_j$ corrisponondo ai pesi che causano sentimento positivo, mentre i pesi negativi corrispondono a sentimento negativo.  

In [18]:
num_positive_weights = len(weights[weights['value'] >= 0])
num_negative_weights = len(weights[weights['value'] < 0])

print "Totale coefficienti positivi: %s " % num_positive_weights
print "Totale coefficienti negativi: %s " % num_negative_weights

Totale coefficienti positivi: 152991 
Totale coefficienti negativi: 48281 


**Quiz question:** How many weights are >= 0?

## Fare previsioni con la logistic regression

Una volta che il modello è addestrato è possibile fare previsioni sui dati di test. In questa sezione prenderemo in considerazione 3 esempi nel dataset di test, che chiameremo **sample_test_data**.

In [14]:
sample_test_data = test_data[207:210]
print sample_test_data['Score']
sample_test_data

[4, 5, 1]


Columns:
	Id	int
	ProductId	str
	UserId	str
	ProfileName	str
	HelpfulnessNumerator	int
	HelpfulnessDenominator	int
	Score	int
	Time	int
	Summary	str
	Text	str
	word_count	dict
	filter_stopwords	dict
	sentiment	int

Rows: 3

Data:
+------+------------+----------------+-------------------------------+----------------------+
|  Id  | ProductId  |     UserId     |          ProfileName          | HelpfulnessNumerator |
+------+------------+----------------+-------------------------------+----------------------+
| 1115 | B003UDV9SG | A2W09PJ1TTHQ6H |         Research Hound        |          0           |
| 1119 | B002N2XXUC | A15FF2P7RPKH6G | Jackquiline D. Jolliffe "J... |          1           |
| 1128 | B000UUYOPW | AGXG6VBHLW8J7  | Danielle M. Karaszkiewicz ... |          1           |
+------+------------+----------------+-------------------------------+----------------------+
+------------------------+-------+------------+--------------------+
| HelpfulnessDenominator | Score |    Time    |      Summary       |
+------------------------+-------+------------+--------------------+
|           0            |   4   | 1342310400 | Good brand of food |
|           2            |   5   | 1306454400 |       lovely       |
|           1            |   1   | 1291248000 |   NOT 6 pieces.    |
+------------------------+-------+------------+--------------------+
+-------------------------------+-------------------------------+
|              Text             |           word_count          |
+-------------------------------+-------------------------------+
| I have now tried both the ... | {'just': 1, 'wasteful': 1,... |
| ordered these for my daugh... | {'ordered': 1, 'days': 1, ... |
| Even though the item says ... | {'refund': 1, 'and': 3, 's... |
+-------------------------------+-------------------------------+
+-------------------------------+-----------+
|        filter_stopwords       | sentiment |
+-------------------------------+-----------+
| {'shedding': 1, 'dont': 1,... |     1     |
| {'ordered': 1, 'overnight'... |     1     |
| {'refund': 1, 'received': ... |     -1    |
+-------------------------------+-----------+
[3 rows x 13 columns]

In [15]:
sample_test_data[0]['Text']

"I have now tried both the Ocean Fish and Tuna and the Duck and Chicken variety of this food.  My cat seems to prefer the taste of the duck and chicken a little better, but I will probably rotate them both because of the health benefits he's gotten (especially from the ocean fish and tuna) that have included a softer coat and less scratching and biting of himself and shedding.  Also, with this food, he doesn't drink much water, but still goes potty alot, so I feel good that he's getting plenty of fluids in his diet.  My only complaint with this brand, and why I only give it 4 stars, is that it has little pieces of potato? and whole peas in it, which my cat eats around and ends up being wasteful, which I don't like given the price.  Not that big of a deal, but it is a minor inconvenience that I have to clean leftover bits out after each feeding as opposed to just washing his empty bowl daily.  (In case you are wondering, it is approximately 4 to 6 whole pea/1.5 pea size potato? bits per

In [16]:
sample_test_data[1]['Text']

'ordered these for my daughters wedding cake overnight so that i would get them the next day<br />they came four days after wedding<br />but they are nice'

In [17]:
sample_test_data[2]['Text']

'Even though the item says "Pack of 6," I only received one. I called them up to see where the other 5 were, and they said that Amazon put it incorrectly. Seriously?? They are issuing a refund, but still. I wanted 6, which is what the item says, and instead and only getting the 1. Should we just not trust what\'s on Amazon now???'

Cerchiamo di prevedere la classe per le review che sono contenute all'interno di  **sample_test_data**. Il `sentiment_model` dà come risultato  **+1** se il sentimento è positivo e **-1** se il sentimento è negativo. Ricorda, dalla lezione precedente che lo **score** (chiamato anche **margin**) per il logistc regression model è definito come:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

dove $h(\mathbf{x}_i)$ rappresenta le features per l'esempio  $i$. 
Per ogni riga **scores** sarà un numero in **[-inf, inf]**.

In [18]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[15.080081706290922, 3.81388694226362, -4.357529571164855]


### Sentiment

Questi scores sono utilizzati per determinare il sentimento nella maniera seguente
$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$



Verifichiamo che le classi predette sono le stesse di quelle ottenute da GraphLab.

In [19]:
print "Classi:" 
print sentiment_model.predict(sample_test_data)

Classi:
[1, 1, -1]



### Previsione basata sulla probabilità

Possiamo calcolare le previsioni in termini di probabilità ,a partire dagli score utilizzando:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$


In [20]:
print sentiment_model.predict(sample_test_data, output_type='probability')

[0.9999997176397188, 0.9784139789923071, 0.012647974287842645]


## Trova le reviews piu positive e piu negative

Analizziamo il completo dataset di test, e utilizziamo GraphLab Create per determinare le previsioni di tutti i prodotti all'interno del dataset di test, con lo scopo di ottenere migliori prestazioni. 

Utilizzando  `sentiment_model`, trova le 20 reviews all'interno di **test_data** con la più **alta probabilità** di essere classificata come **review positiva**. 
Chiameremo queste reviews **most positive reviews**.

Come facciamo a calcolare queste 20 reviews?
Seguiamo i seguenti passi: 
1.  Calcola le previsioni tramite la probabilità sui dati di test usando`sentiment_model` 
2.  Ordina i dati secondo queste predizioni e prendi i primi 20.

In [21]:
test_data['prediction'] = sentiment_model.predict(test_data, output_type='probability')

In [22]:
top_20_positive = test_data.topk('prediction', k=20)
top_20_positive.print_rows(num_rows=20)

+-------+------------+----------------+-------------------------------+----------------------+
|   Id  | ProductId  |     UserId     |          ProfileName          | HelpfulnessNumerator |
+-------+------------+----------------+-------------------------------+----------------------+
|  3022 | B000B9SFP2 | AZFHSPEZUPGD2  |            Carol M            |          4           |
|  3025 | B000B9SFP2 | A3U78OW4OO2Q8W |      poltroon "poltroon"      |          0           |
| 11520 | B0002UM1AA | A3072VKAJZ18X0 |             Meglet            |          1           |
| 12241 | B00141UC9I | A1EUHMS0VCAX31 |      Karen "kawaiioni02"      |          6           |
| 18638 | B00008JOL0 | AWOR961LMDUIF  |            J. Davis           |          3           |
| 18921 | B000F4H5FE | A30JGTOA15XB8W |          D. J. Smith          |          4           |
| 19571 | B000084ETV | A3NTV9VHMBY1KH | Kimberly A. Kubek "Doodle ... |          9           |
| 19828 | B00390T5JA | A2TJG4N8LNJW23 |         Bl

In [23]:
top_20_positive[15]["Text"]

"*Note to Amazon.com,<br /><br />Pretty please put this Liver on Subscribe and Save! You have the beef & liver and the beef available for S&S but not the Liver yet. THANK YOU SO MUCH!<br /><br />Now on to my review...<br /><br />My cat has been slowly switching to Newman's own beef products over from Wellness brand cat food (which is a great brand and will still be reserved for treats occasionally). I was ecstatic that there was a totally NO grain-and-NO fillers ORGANIC cat food I could feed my 9 year old cat who is prone to crystal formation in his bladder and also plaque build up on his teeth (even though I brush his outer teeth every evening).<br /><br />So far I've tried the beef & Liver and the Liver and he LOVES it. I just ordered the beef today for him which I'm sure he'll love too.<br /><br />This product *IS* very thick and concentrated (the beef liver alone has the consistency of a concentrated pate' and the beef & liver has the consistency of a gelatinous, thick beefy oatmea

In [24]:
top_20_negative = test_data.topk('prediction', k=20, reverse=True)
top_20_negative.print_rows(num_rows=20)

+--------+------------+----------------+----------------------------+----------------------+
|   Id   | ProductId  |     UserId     |        ProfileName         | HelpfulnessNumerator |
+--------+------------+----------------+----------------------------+----------------------+
| 448127 | B003MOXA6C | A3FOM5AA39AAS0 |    Vasiliki "vasiliki"     |          1           |
| 226249 | B002T0NWKE | A33ZYFE8XMKKR1 | P. Scott Pope "Scott Pope" |          31          |
| 338239 | B002T0GXNC | A33ZYFE8XMKKR1 | P. Scott Pope "Scott Pope" |          31          |
| 177656 | B003561V40 | A33ZYFE8XMKKR1 | P. Scott Pope "Scott Pope" |          31          |
| 354818 | B003YV46WK | A33ZYFE8XMKKR1 | P. Scott Pope "Scott Pope" |          31          |
| 219822 | B004SKZWBK | AFAWC4ZJ29YV1  |             AG             |          0           |
| 325920 | B000OH73FO | A33YMBT1Y895S  |            Kara            |          2           |
| 318796 | B003MW7858 | A14VQRIH3FDX5K |         kittenkatt         | 

## Calcolo accuracy del classificatore

L'accuracy può esssere calcolata come:

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

Seguiamo i seguenti passi per costruire il metodo get_accuracy:

* **Passo 1:** Usa il modello addestrato per calcolare le predizioni delle classi. 
* **Passo 2:** Conta il numero di osservazioni dove la classe prevista è correttamente classificata(ha la stessa etichetta della classe vera).
* **Passo 3:** Dividi il totale delle previsioni corrette per il totale delle osservazioni nel dataset. 



In [ ]:
def get_accuracy(model, data, true_labels):
    # Passo 1: calcolo le previsioni delle classi
    predictions = model.predict(data)
    
    # Passo 2 : conto il numero di previsioni corrette
    num_correct = sum(predictions == true_labels)

    # Calcolo accuracy dividendo le previsioni corrette per
    # il numero totale delle osservazioni
    accuracy = num_correct / len(data)
    
    return accuracy

Ora calcoliamo l'accuracy di **sentiment_model** sui dati di test

In [ ]:
get_accuracy(sentiment_model, test_data, test_data['sentiment'])

## Addestra un altro classificatore con poche parole 

Ci sono moltissime parole nel modello che abbiamo addestrato in precedenza. 
Addestriamo un classificatore più semplice utilizzando soltanto un sottoinsieme delle parole che si trovano nelle reviews.

In [ ]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able','less', 'even', 'waste', 'disappointed', 'money', 'would', 
                     'return','disgusting','yummy','cool','acid','beautiful','baked','chocolate',
                     'homemade', 'low-fat','nutty','pickled','plain','dry','fantastic',
                     'delicious','delectable'
                    ,'delightful','fantastic','flavored','flavorless','tasteful','tasteless'
                    'tasty','tender','smelly','smells','rancid','small','fried','insipid','low','well']

In [ ]:
len(significant_words)

Per ogni review, utilizziamo la colonna **word_count** e rimuoviamo le parole che non si trovano nella lista delle parole significative. Faremo questa cosa sia sui dati di train che di test

In [ ]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys (significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Vediamo il primo esempio:

In [ ]:
train_data[100]['Text']

In [ ]:
print train_data[100]['filter_stopwords']

La colonna **word_count_subset** è un sottoinsieme del dizionario, in questo esempio solo   1 `significant words` è presente nella review.

In [ ]:
print train_data[100]['word_count_subset']

In [ ]:
## Addestra il modello sul sottoinsieme dei dati

Costruiamo un classificatore con word_count_subset come feature e sentiment come target.

In [ ]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Calcoliamo l'accuracy su questo nuovo modello:

In [ ]:
get_accuracy(simple_model, test_data, test_data['sentiment'])

Analizziamo i coefficienti di questo modello

In [ ]:
simple_model.coefficients

Ordiniamo i coefficienti in ordine decrescente per ottenere i coefficienti con gli effetti più positivi sul sentiment.

In [ ]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=47)

In [ ]:
len(simple_model.coefficients[simple_model.coefficients['value'] > 0])

In [ ]:
simple_model.coefficients[simple_model.coefficients['value'] >= 0]

In [ ]:
weights[weights['index'] == 'tasteful']

# Confrontare i modelli

Calcoliamo l'accuracy del  **sentiment_model** e del **simple_model** utilizzando il metodo `get__accuracy` implementato sopra

Calcoliamo l'accuracy per **sentiment_model** sui dati di train

In [ ]:
get_accuracy(sentiment_model, train_data, train_data['sentiment'])

Calcoliamo l'accuracy per **simple_model** sui dati di train

In [ ]:
get_accuracy(simple_model, train_data, train_data['sentiment'])

Quale modello ha accuracy più alta sui dati di train?

Ora facciamo la stessa cosa sui dati di test:

In [ ]:
get_accuracy(sentiment_model, test_data, test_data['sentiment'])

In [ ]:
get_accuracy(simple_model, test_data, test_data['sentiment'])

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

E’ comune utilizzare un majority class classifier come modello di riferimento da comparare con il nostro. 
Il majority classifier prevede per tutte le osservazioni la classe il cui insieme ha un maggior numero di elementi.
E’ necessario ottenere un punteggio maggiore all’approccio baseline, in modo che il modello abbia senso
Qual è la majority class per i nostri dati di train?

In [20]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

Calcolo, per ognuna delle classi all’interno dei dati di train il numero di osservazioni corrispondente. 

In [ ]:
print float((test_data['sentiment'] == +1).sum()) / len(test_data)